In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw

import sys
sys.path.append('..')
from detector.input_pipeline import Pipeline
from params import wider_params

# Build a graph

In [ ]:
tf.reset_default_graph()

pipeline = Pipeline(
    ['/mnt/datasets/COCO/train_shards/shard-0000.tfrecords'],
    is_training=True, params=wider_params
)

dataset = pipeline.dataset
iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
init = iterator.make_initializer(dataset)
features, labels = iterator.get_next()
features.update(labels)  # for convenience

# Show an augmented image with boxes

In [ ]:
with open('coco_labels.txt', 'r') as f:
    label_encoder = {line.strip(): i for i, line in enumerate(f.readlines()) if line.strip()}

label_decoder = {i: n for n, i in label_encoder.items()}

In [ ]:
def draw_boxes(image, boxes, labels):
    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy, 'RGBA')

    for box, label in zip(boxes, labels):
        ymin, xmin, ymax, xmax = box

        fill = (255, 0, 0, 75)
        outline = 'black'

        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
        draw.text((xmin + 2, ymin), label_decoder[label])

    return image_copy

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output = sess.run(features)

image = Image.fromarray((255.0*output['images'][0]).astype('uint8'))
boxes = output['boxes'][0]
classes = output['labels'][0]

print(image.size)
draw_boxes(image, boxes, classes)